# Create Json mask file to choose the appropriate threshold

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os, sys

other_dir_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(other_dir_path)

print(other_dir_path)
import IMCprocess.spatial_analysis.spatial_analysis as sa
import IMCprocess.spatial_analysis.combi_enrich as ce
import IMCprocess.utils.utils as ut

# I have to use custom Img_anndata code
import Img_anndata as utan

/Users/pattama/Desktop/UiB/cancer/git_repo/IMCprocess


## Import mask and level1 annotated object and create epithelial object

In [2]:
data_dir = '/Users/pattama/Desktop/UiB/cancer/example_share_code/share_with_vladan/mask_analysis_full_pipeline/example_data/'
input_mask = data_dir + 'Patient2_001.tiff'
output_dir = 'output'
level1_file = data_dir + '00hanna_adata_totalNB_obj_Level1.pkl'
myimg_id = 'R2_001'
Img_obj_level1 = ut.read_pickle_obj(level1_file)
# Get adata level1 for img_id R1_001
Img_level1_R2_001 = ut.filter_adata_obs(Img_obj_level1.adata_dict['level1'],
                                        'img_id', 
                                        [myimg_id])
# Get Epi adata
Epi_adata = ut.filter_adata_obs(Img_level1_R2_001,
                                'level1_annotated',
                                ['Ep'])

In [3]:
# Check marker column name
Epi_adata.var_names

Index(['79Br', '80ArAr', '81Br', '89Y', '113In_ADHL1', '115In_GammaCat',
       '127I', '131Xe', '138Ba', '139La_MCAM', '141Pr_aSMA', '142Nd_CTLA4',
       '143Nd_Vimentin', '144Nd_YAP1', '145Nd_CD45', '146Nd_CD16',
       '147Sm_CD163', '148Nd_PanKeratin', '149Sm_CK5_6', '150Nd_PDL1',
       '151Eu_CD31_CD34', '152Sm_FAP', '153Eu_CD44', '154Sm_CD11c',
       '155Gd_FOXP3', '156Gd_CD4', '158Gd_E_Cad', '159Tb_CD68', '160Gd_PDGFRa',
       '161Dy_pSMAD2', '162Dy_CD8', '163Dy_ER', '164Dy_CD20', '165Ho_PD1',
       '166Er_HER2', '167Er_GATA3', '168Er_Ki67', '169Tm_CD24', '170Er_CD3',
       '171Yb_PDGFRb', '172Yb_CC3', '173Yb_GranzymeB', '174Yb_CK8_18',
       '175Lu_PanActin', '176Yb_HisH3', '191Ir_DNA', '193Ir_DNA', '195Pt_mem1',
       '196Pt_mem2', '198Pt_mem3', '209Bi_pSTAT3'],
      dtype='object')

## label 1, 0 for positive and negative cells from threshold

In [4]:
# User provide input marker column and 
# range of threshold for each marker
marker_threshold_dict = {'168Er_Ki67': [0.2, 0.3, 0.4, 0.5],
                         '143Nd_Vimentin': [0.2, 0.3, 0.4, 0.5],
                         '169Tm_CD24': [0.2, 0.3, 0.4, 0.5]
}

In [5]:
def binary_df(df, threshold):
    df_binary = df.applymap(lambda x: 1 if x >= threshold else 0)
    df_binary.columns = [col + '_'+ str(round(threshold,2)) for col in df.columns]
    return(df_binary)

def cell_neg_pos_dict(df, col):
    cell_neg_ind = np.array(df[df[col] == 0].index) + 1
    cell_pos_ind = np.array(df[df[col] == 1].index) + 1
    label_dict = {('neg_'+col): cell_neg_ind,
                  ('pos_'+col): cell_pos_ind}
    return(label_dict)

In [6]:
def create_json_marker(input_mask, input_adata, marker_col, 
                       threshold, output_dir):
    '''
    Create json file for specific marker and threshold
    '''
    # 1. get marker expression df
    marker_df = input_adata.to_df()[[marker_col]]
    # 2. get binary df
    bi_df = binary_df(marker_df, threshold).reset_index()
    # 3. get cell label dict
    cell_label_dict = cell_neg_pos_dict(bi_df, 
                                        marker_col + '_'+ str(round(threshold,2)))
    # 4. Check if output dir exist and crate mask
    if not os.path.exists(output_dir+ marker_col):
        os.makedirs(output_dir+ marker_col)
        print(f"{output_dir+marker_col} created successfully!")

    ut.export_labelmask_geoJson(input_mask, 
                                output_dir+ '/'+ marker_col + '/' + 
                                str(round(threshold,2)).replace('.','_') + '.json',
                                cell_label_dict,
                                [-1, -8245601]
                                )

In [7]:
for marker, threshold_list in marker_threshold_dict.items():
    print('create mask for ' + marker + 
          ' with threshold ' + str(threshold_list))
    for threshold in threshold_list:
        create_json_marker(input_mask, 
                           Epi_adata, 
                           marker, 
                           threshold, output_dir+ '/Json_mask/')

create mask for 168Er_Ki67 with threshold [0.2, 0.3, 0.4, 0.5]
create mask for 143Nd_Vimentin with threshold [0.2, 0.3, 0.4, 0.5]
output/Json_mask/143Nd_Vimentin created successfully!
create mask for 169Tm_CD24 with threshold [0.2, 0.3, 0.4, 0.5]
output/Json_mask/169Tm_CD24 created successfully!
